In [1]:
import numpy as np
import pandas as pd
from scipy import stats

# Dane z Twojego obrazka
yamaha = [14, 7, 23, 16, 18, 16, 19, 17, 17, 24, 18, 16, 20, 12, 16, 16, 16, 25, 19, 14]
suzuki = [20, 20, 18, 14, 23, 16, 18, 16, 20, 11, 19, 20, 24, 24, 17, 22, 13, 22, 18, 23]

# --- 1. Założenie: Niezależność ---
# Z treści zadania wiemy, że to dwie różne grupy osób - zaznaczamy checkbox o niezależności.

# --- 2. Test Normalności (Shapiro-Wilk) ---
shapiro_y = stats.shapiro(yamaha)
shapiro_s = stats.shapiro(suzuki)

# --- 3. Test Homogeniczności Wariancji (Levene) ---
levene_test = stats.levene(yamaha, suzuki)

# --- 4. Test t-Studenta ---
# Używamy equal_var=True, jeśli p-value Levene'a > 0.05
equal_v = levene_test.pvalue > 0.05
t_stat, p_val = stats.ttest_ind(yamaha, suzuki, equal_var=equal_v)

print(f"Normalność Yamaha (p): {shapiro_y.pvalue:.4f}")
print(f"Normalność Suzuki (p): {shapiro_s.pvalue:.4f}")
print(f"Wariancje (p): {levene_test.pvalue:.4f}")
print("-" * 20)
print(f"Test Statystyczny (t): {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")

Normalność Yamaha (p): 0.2298
Normalność Suzuki (p): 0.4658
Wariancje (p): 0.9517
--------------------
Test Statystyczny (t): -1.4284
p-value: 0.1613


In [3]:
import scipy.stats as stats

# Dane z obrazka
t_stat = -7.3874
df = 111
p_value = 2.973e-11
mean_sparag = 3.798223
mean_consumo = 7.603088
alpha = 0.05

print("--- Analiza wyników testu t ---")

# a. Czy wartość bezwzględna statystyki testowej jest większa niż 1.96?
abs_t = abs(t_stat)
print(f"a) |t| = {abs_t:.4f} > 1.96: {abs_t > 1.96}")

# b. Czy test był jednostronny? 
# W tekście "alternative hypothesis: true difference... is not equal to 0" 
# wskazuje na test dwustronny (two-sided).
is_one_sided = False 
print(f"b) Czy test był jednostronny?: {is_one_sided}")

# c. Czy p-value jest większe niż 0.05?
print(f"c) p-value ({p_value}) > 0.05: {p_value > alpha}")

# d i e. Porównanie średnich czasu oczekiwania
print(f"d) Sparag ({mean_sparag:.2f}) > Consumo ({mean_consumo:.2f}): {mean_sparag > mean_consumo}")
print(f"e) Sparag ({mean_sparag:.2f}) < Consumo ({mean_consumo:.2f}): {mean_sparag < mean_consumo}")

# Dodatkowa weryfikacja p-value na podstawie t i df
calculated_p = stats.t.sf(abs_t, df) * 2
print(f"\nObliczone p-value na podstawie t i df: {calculated_p:.2e}")

--- Analiza wyników testu t ---
a) |t| = 7.3874 > 1.96: True
b) Czy test był jednostronny?: False
c) p-value (2.973e-11) > 0.05: False
d) Sparag (3.80) > Consumo (7.60): False
e) Sparag (3.80) < Consumo (7.60): True

Obliczone p-value na podstawie t i df: 2.97e-11


In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.power import NormalIndPower

# 1. Wczytanie danych
# Zmień 'dataset.csv' na nazwę swojego pliku
df = pd.read_csv('dataset (2).csv')

# 2. Przygotowanie danych
# Filtrujemy dane, aby usunąć brakujące wartości (jak prosi polecenie)
df = df[['Sex', 'Smoke']].dropna()

# Definiujemy grupy
males = df[df['Sex'] == 'Male']
females = df[df['Sex'] == 'Female']

# Liczymy osoby palące "occasionally"
successes = np.array([
    (males['Smoke'] == 'Occas').sum(),
    (females['Smoke'] == 'Occas').sum()
])

# Liczymy łączną liczbę studentów w każdej grupie
nobs = np.array([len(males), len(females)])

# 3. Test Statystyczny (Z-test dla proporcji)
# alpha = 0.01
# alternative='larger', bo sprawdzamy czy mężczyźni palą częściej niż kobiety
stat, p_value = proportions_ztest(successes, nobs, alternative='larger')

print(f"Statystyka testowa: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 4. Obliczanie wymaganej wielkości próby dla mocy 90% (0.9)
# Najpierw obliczamy aktualne proporcje i wielkość efektu (Effect Size)
p1 = successes[0] / nobs[0]
p2 = successes[1] / nobs[1]

# Obliczamy Cohen's h (miara wielkości efektu dla proporcji)
h = 2 * (np.arcsin(np.sqrt(p1)) - np.arcsin(np.sqrt(p2)))

analysis = NormalIndPower()
required_n = analysis.solve_power(effect_size=h, alpha=0.01, power=0.9, ratio=1, alternative='larger')

print(f"Wymagana liczba respondentów (na jedną grupę): {np.ceil(required_n)}")
print(f"Łączna wymagana liczba respondentów: {np.ceil(required_n) * 2}")

Statystyka testowa: 0.6728
p-value: 0.2505
Wymagana liczba respondentów (na jedną grupę): 3475.0
Łączna wymagana liczba respondentów: 6950.0
